# **Job Matching WBL for Mentee-Mentor**

# Use Case Documentation

**Description :**
Merekomendasi role/posisi paling cocok untuk peserta magang (mentee) dalam program WBL (Work-Based Learning) berbasis data profil mentee, sehingga bisa dicocokkan secara tepat dengan kebutuhan mentor atau supervisor.

**Actors**
1. Supervisor (Mentor)
2. Mentee

**Main Flow**
1. Mentee memasukkan data profil dan skill, mentor memasukkan job request  
2. Sistem memproses data mentee dan mencocokkan dengan role yang tersedia  
3. Sistem menampilkan rekomendasi role yang paling cocok  
4. Supervisor dapat melihat hasil rekomendasi


**Alternative Flow :**
Jika tidak ditemukan role yang cocok secara otomatis, sistem menyediakan opsi pencarian manual agar mentor tetap dapat memilih mentee sesuai role yang dibutuhkan.

**Expected Outcome :**
Mentor dapat dengan mudah menyeleksi mentee berdasarkan rekomendasi yang sesuai dengan kebutuhan skill pada posisi yang ditawarkan, sehingga proses seleksi menjadi lebih efisien.


# Data Understanding

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import ast

In [ ]:
# Getting each json into csv

files = [
    "*/dataset/allReqParticipant.json",
    "*/dataset/groupByReserve.json",
    "*/dataset/groupBySpv.json",
]

output_loc = "*/transformed_src/"

for file in files:
    with open(file) as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        output_csv = os.path.join(output_loc, os.path.basename(file).replace(".json", ".csv"))
        df.to_csv(output_csv, index=False)

In [ ]:
df_allReqParticipant = pd.read_csv(output_loc + "allReqParticipant.csv")
df_groupByReserve = pd.read_csv(output_loc + "groupByReserve.csv")
df_groupBySpv = pd.read_csv(output_loc + "groupBySpv.csv")

list = [df_allReqParticipant, df_groupByReserve, df_groupBySpv]

for i in list:
    print(f"{i.info()}\n")

In [ ]:
df_groupByReserve

In [ ]:
df_allReqParticipant

In [ ]:
df_groupBySpv

# Data Preparation

### Normalizing the data

In [ ]:
# Function to extract role data

def normalize_nested_column(df, column_name, explode_column=True):
    # Handle missing values
    df[column_name] = df[column_name].fillna('{}').apply(ast.literal_eval)
    
    if explode_column:
        df = df.explode(column_name).reset_index(drop=True)
    
    normalized = pd.json_normalize(df[column_name])
    df = pd.concat([df.drop(column_name, axis=1), normalized], axis=1)
    return df

In [ ]:
df_req = pd.read_csv('*/transformed_src/allReqParticipant.csv', parse_dates=['createdAt', 'updatedAt'])
df_req = normalize_nested_column(df_req, 'roleRequest')

df_reserve = pd.read_csv('*/transformed_src/groupByReserve.csv')
df_reserve = normalize_nested_column(df_reserve, 'participants')

df_spv = pd.read_csv('*/transformed_src/groupBySpv.csv')
df_spv = normalize_nested_column(df_spv, 'participants')


In [ ]:
col_names_reserve = ['reserve_id', 'reserve_name', 'participant_id', 'participant_name']
col_names = ['spv_id', 'spv_name', 'participant_id', 'participant_name']

for i in range(4):
    df_reserve.columns.values[i] = col_names_reserve[i]
    df_spv.columns.values[i] = col_names[i]

In [ ]:
print(df.isnull())

In [ ]:
df_req

In [ ]:
print(df_req.columns)

In [ ]:
df_req['jobDesc']

In [ ]:
df_req

In [ ]:
df_req_new = df_req.copy()

In [ ]:
column_list = df_req_new.columns.tolist()
print(column_list)

In [ ]:
columns_to_drop = [
'status', 'projectName', 'projectLocation', 'projectDesc', 'projectLink', 'batchType', 'batchTypeName', 'batchRequestQuota', 'amount', 'createdAt', 'updatedAt', 'internshipPeriodMonth', 'internshipPeriodMonthStart', 'internshipPeriodMonthEnd', 'onJobScheme', 'universityRequest', 'isUniversityRequest', 'internshipPeriodStartDate', 'internshipPeriodEndDate',]

df_req_new.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df_req_new.rename(columns={'supervisorName': 'spv_name'}, inplace=True)

In [ ]:
df_req_new

In [ ]:
df_req_new.isna().sum()

In [ ]:
df_req_new

In [ ]:
print(df_req.dtypes)

In [ ]:
df_reserve

In [ ]:
df_reserve.to_csv('*/transformed_src/df_reserve.csv', index=False)

In [ ]:
df_rsv = pd.read_csv('*/transformed_src/df_reserve.csv')
df_rsv = normalize_nested_column(df_rsv, 'curriculumVitae.educationalBackgrounds')
df_rsv = normalize_nested_column(df_rsv, 'curriculumVitae.workExperiences')


In [ ]:
df_rsv

In [ ]:
column_list = df_rsv.columns.tolist()
print(column_list)

In [ ]:
columns_to_drop = [
    'curriculumVitae.englishSkill.certificationType', 'curriculumVitae.englishSkill.certificationPublisher', 'curriculumVitae.englishSkill.certificateNumber', 'curriculumVitae.englishSkill.publishDate', 'curriculumVitae.skill.hardSkillDesc', 'curriculumVitae.englishSkill.expireDate', 'curriculumVitae.englishSkill.score', 'curriculumVitae.organizationExperiences', 'curriculumVitae.skill.softSkill', 'curriculumVitae.finalTask.title', 'curriculumVitae.finalTask.status', 'curriculumVitae.finalTask.desc', 'curriculumVitae.achievements', 'curriculumVitae.socialMedia.linkedIn', 'curriculumVitae.socialMedia.instagram', 'curriculumVitae.socialMedia.twitter', 'curriculumVitae.socialMedia.youtube', 'curriculumVitae.portfolio.portfolioLink', 'curriculumVitae.portfolio.creativeCVFileName', 'curriculumVitae.generatedCVFileName', 'curriculumVitae.finalTask.chapterProgress', 'curriculumVitae.finalTask.totalChapter', 'curriculumVitae.photoCVFileName', 'curriculumVitae.projectExperiences', 'curriculumVitae.skill.areaOfInterest', 'curriculumVitae.skill.hardSkillLevel', 'curriculumVitae.certification', 'curriculumVitae.trainingCertification', 'educationLevel', 'institutionName', 'institutionCity', 'department', 'ipk', 'status', 'semester', 'type', 'companyName', 'startDate', 'endDate', 'isActive'
]

df_rsv.drop(columns=columns_to_drop, inplace=True)


In [ ]:
df_rsv

In [ ]:
df_rsv.isna().sum()

In [ ]:
df_rsv.dropna(inplace=True)

In [ ]:
df_rsv

In [ ]:
df_spv

In [ ]:
df_spv.to_csv('*/transformed_src/df_spv.csv', index=False)

In [ ]:
df_supervisior = pd.read_csv('*/transformed_src/df_spv.csv')
df_supervisior = normalize_nested_column(df_supervisior, 'curriculumVitae.educationalBackgrounds')
df_supervisior = normalize_nested_column(df_supervisior, 'curriculumVitae.workExperiences')

In [ ]:
df_supervisior

In [ ]:
column_list = df_supervisior.columns.tolist()
print(column_list)

In [ ]:
columns_to_drop = [
    'curriculumVitae.photoCVFileName', 'curriculumVitae.englishSkill.certificationType', 'curriculumVitae.englishSkill.certificationPublisher', 'curriculumVitae.englishSkill.certificateNumber', 'curriculumVitae.englishSkill.publishDate', 'curriculumVitae.englishSkill.expireDate', 'curriculumVitae.englishSkill.score', 'curriculumVitae.organizationExperiences', 'curriculumVitae.skill.hardSkillDesc', 'curriculumVitae.projectExperiences', 'curriculumVitae.skill.areaOfInterest', 'curriculumVitae.skill.hardSkillLevel', 'curriculumVitae.skill.softSkill', 'curriculumVitae.finalTask.title', 'curriculumVitae.finalTask.status', 'curriculumVitae.finalTask.chapterProgress', 'curriculumVitae.finalTask.totalChapter', 'curriculumVitae.finalTask.desc', 'curriculumVitae.achievements', 'curriculumVitae.certification', 'curriculumVitae.trainingCertification', 'curriculumVitae.socialMedia.linkedIn', 'curriculumVitae.socialMedia.instagram', 'curriculumVitae.socialMedia.twitter', 'curriculumVitae.socialMedia.youtube', 'curriculumVitae.portfolio.portfolioLink', 'curriculumVitae.portfolio.creativeCVFileName', 'curriculumVitae.generatedCVFileName', 'educationLevel', 'institutionName', 'institutionCity', 'department', 'ipk', 'status', 'semester', 'type', 'companyName', 'startDate', 'endDate', 'isActive'
]

df_supervisior.drop(columns=columns_to_drop, inplace=True)


In [ ]:
df_supervisior

In [ ]:
df_supervisior.isna().sum()

In [ ]:
df_supervisior.dropna(inplace=True)

In [ ]:
df_supervisior

### Merging the DataFrames (df_supervisior and df_rsv)

In [ ]:
df_mentee_participant = pd.merge(df_supervisior, df_rsv, on='participant_id', how='outer')



In [ ]:
df_mentee_participant

### Merging the DataFrames (df_req_new and df_mentee_participant)

In [ ]:
df_final_participant = pd.merge(df_req_new, df_mentee_participant, on='spv_name', how='outer')


In [ ]:
df_final_participant

In [ ]:
df_final_participant.isna().sum()

In [ ]:
df_final_participant

### Renaming Columns in df_final_participan

In [ ]:
df_final_participant.rename(columns={
    "_id": "request_id",
    "supervisor_nik": "supervisor_nik",
    "spv_name": "supervisor_name",
    "jobPosition": "job_position",
    "jobDesc": "job_description",
    "jobSpec": "job_specification",
    "tools": "required_tools",
    "skills": "required_skills",
    "internshipRole._id_x": "internship_role_id",
    "internshipRole.title_x": "internship_role_title",
    "spv_id": "supervisor_id",
    "participant_id": "participant_id",
    "participant_name_x": "participant_name",
    "acceptedInternshipRole._id": "accepted_role_id",
    "acceptedInternshipRole.title": "accepted_role_title",
    "curriculumVitae.skill.hardSkillCategory_x": "participant_skill",
    "curriculumVitae.skill.tools_x": "participants_tools",
    # "curriculumVitae.skill.hardSkillDesc_x": "participant_specification",
    "position_x": "internship_position",
    "description_x": "internship_description",
    "reserved_id": "reserved_id",
    "reserve_name": "reserved_name",
    "participant_name_y": "booked_participant_name",
    "internshipRole._id_y": "booked_role_id",
    "internshipRole.title_y": "booked_role_title",
    "curriculumVitae.skill.tools_y": "booked_tools",
    "curriculumVitae.skill.hardSkillCategory_y": "booked_skill",
    # "curriculumVitae.skill.hardSkillDesc_y": "booked_specification",
    "position_y": "booked_position",
    "description_y": "booked_description"
}, inplace=True)


In [ ]:
df_final_participant

In [ ]:
print(df_final_participant.describe)

In [ ]:
import pandas as pd
import os

# Lokasi output
output_loc = "*/transformed_src/"
output_file = os.path.join(output_loc, "df_final_participant.xlsx")

# Pastikan folder tujuan ada
if not os.path.exists(output_loc):
    os.makedirs(output_loc)

# Simpan ke Excel
df_final_participant.to_excel(output_file, index=False)


print(f"File berhasil disimpan di {output_file}")

### Grouping and Aggregating Final Participant Data by Participant Name

In [ ]:
df_merged_final = df_final_participant.groupby(['participant_name'], as_index=False).agg({
    'internship_position': lambda x: '\n'.join(
        f"{i+1}. {pos}" for i, pos in enumerate(sorted(x.fillna('Tidak ada data').unique()))
    ),
    'booked_position': lambda x: '\n'.join(
        f"{i+1}. {pos}" for i, pos in enumerate(sorted(x.fillna('Tidak ada data').unique()))
    ),
})


In [ ]:
df_merged_final

### Merging the DataFrames (df_final_participant and df_merged_final)

In [ ]:
df_fix_participant = pd.merge(df_final_participant, df_merged_final, on='participant_name', how='outer')


In [ ]:
df_fix_participant

### Cleaning and Renaming Columns in df_fix_participant

In [ ]:

df_fix_participant['booked_position_x'] = df_fix_participant['booked_position_y']


df_fix_participant['internship_position_x'] = df_fix_participant['internship_position_y']


df_fix_participant = df_fix_participant.drop(columns=['booked_position_y', 'internship_position_y'])


df_fix_participant = df_fix_participant.rename(columns={'booked_position_x': 'booked_position', 
                                                        'internship_position_x': 'internship_position'})


In [ ]:
df_fix_participant

In [ ]:
column_list = df_fix_participant.columns.tolist()
print(column_list)

In [ ]:
columns_to_drop = [
    'job_description', 'internship_description', 'booked_description'
]

df_fix_participant.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df_fix_participant

In [ ]:
column_list = df_fix_participant.columns.tolist()
print(column_list)

### Filtering and Dropping Rows in df_fix_participant

In [ ]:
import pandas as pd

# List kolom yang harus kosong
kolom_kosong = [
    'request_id', 'supervisor_nik', 'job_position', 'job_specification',
    'required_tools', 'required_skills', 'internship_role_id', 'internship_role_title'
]

# List kolom yang harus terisi
kolom_terisi = [
    'supervisor_name', 'supervisor_id', 'participant_id', 'participant_name',
    'accepted_role_id', 'accepted_role_title', 'participant_skill', 'participants_tools',
    'internship_position', 'reserve_id', 'reserved_name', 'booked_participant_name',
    'booked_role_id', 'booked_role_title', 'booked_skill', 'booked_tools', 'booked_position'
]

# Filter baris sesuai kondisi
drop_cases = df_fix_participant[
    df_fix_participant[kolom_kosong].isna().all(axis=1) &
    df_fix_participant[kolom_terisi].notna().all(axis=1)
]

# Menampilkan jumlah baris yang di-drop
print(f"Jumlah baris yang di-drop: {len(drop_cases)}")

# Hapus baris-baris tersebut dari df_fix_participant
df_cleaned1 = df_fix_participant.drop(drop_cases.index)

# Jika ingin mengecek hasil, bisa ditampilkan
print(df_cleaned1.head())


In [ ]:
df_cleaned1

In [ ]:
import pandas as pd

# List kolom yang harus kosong
kolom_kosong = [
    'request_id', 'supervisor_nik', 'job_position', 'job_specification',
    'required_tools', 'required_skills', 'internship_role_id', 'internship_role_title',
    'reserve_id', 'reserved_name', 'booked_participant_name', 'booked_role_id',
    'booked_role_title', 'booked_skill', 'booked_tools'
]

# Kolom yang harus terisi
kolom_terisi = [
    'supervisor_id', 'supervisor_name', 'participant_id', 'participant_name',
    'accepted_role_id', 'accepted_role_title', 'participant_skill', 'participants_tools',
    'internship_position'
]

# Filter baris yang memenuhi kondisi: request dan booking kosong, booked_position = "1. Tidak ada data"
drop_cases = df_cleaned1[
    df_cleaned1[kolom_kosong].isna().all(axis=1) & 
    (df_cleaned1['booked_position'] == "1. Tidak ada data") &
    df_cleaned1[kolom_terisi].notna().all(axis=1)
]

# Menampilkan jumlah baris yang di-drop
print(f"Jumlah baris yang di-drop: {len(drop_cases)}")

# Hapus baris-baris tersebut dari df_cleaned1
df_cleaned2 = df_cleaned1.drop(drop_cases.index)

# Menampilkan hasil jika ingin dicek
print(df_cleaned2.head())


In [ ]:
df_cleaned2

In [ ]:
import pandas as pd

# Daftar kolom yang HARUS kosong (booking & onjob)
kolom_kosong = [
    'supervisor_id', 'participant_name', 'participant_id',
    'accepted_role_id', 'accepted_role_title', 'participant_skill',
    'participants_tools', 'internship_position',
    'reserve_id', 'reserved_name', 'booked_participant_name',
    'booked_role_id', 'booked_role_title', 'booked_skill',
    'booked_tools', 'booked_position'
]

# Daftar kolom yang HARUS terisi (request lengkap)
kolom_terisi = [
    'supervisor_name', 'request_id', 'supervisor_nik',
    'job_position', 'job_specification', 'required_tools',
    'required_skills', 'internship_role_id', 'internship_role_title'
]

# Filter baris yang memenuhi kondisi tersebut
drop_cases = df_cleaned2[
    df_cleaned2[kolom_kosong].isna().all(axis=1) &  # semua kolom_kosong harus NaN
    df_cleaned2[kolom_terisi].notna().all(axis=1)   # semua kolom_terisi harus terisi
]

# Tampilkan jumlah baris yang akan di-drop
print(f"Jumlah baris yang di-drop: {len(drop_cases)}")

# Buat DataFrame baru dengan baris-baris tersebut dihapus
df_fix_cleaned = df_cleaned2.drop(drop_cases.index)

# Opsional: tampilkan 5 baris pertama untuk cek hasil
print(df_fix_cleaned.head())


In [ ]:
df_fix_cleaned

In [ ]:
df_fix_participant_cleaned = df_fix_cleaned

In [ ]:
# Kolom yang harus terisi agar status = 'diterima'
request_and_on_job_cols = [
    'request_id', 'supervisor_nik', 'supervisor_name', 'job_position', 'job_specification',
    'required_tools', 'required_skills', 'internship_role_id', 'internship_role_title',
    'supervisor_id', 'participant_id', 'participant_name', 'accepted_role_id',
    'accepted_role_title', 'participant_skill', 'participants_tools', 'internship_position'
]

def set_status(row):
    if row[request_and_on_job_cols].notna().all():
        return 'diterima'
    else:
        return 'ditolak'

# Tambahkan kolom status
df_fix_participant_cleaned['status'] = df_fix_participant_cleaned.apply(set_status, axis=1)

# Kosongkan participant_id jika status ditolak
df_fix_participant_cleaned.loc[df_fix_participant_cleaned['status'] == 'ditolak', 'participant_id'] = None

In [ ]:
df_fix_participant_cleaned

In [ ]:
booked_reserved_df = df_fix_participant_cleaned[[
    'booked_participant_name',
    'booked_skill',
    'booked_tools',
    'booked_role_title',
    'booked_position',
    'reserved_name'
    
]].copy()


booked_reserved_df = booked_reserved_df.rename(columns={
    'booked_participant_name': 'participant_name',
    'booked_skill': 'participant_skill',
    'booked_tools': 'participants_tools',
    'booked_role_title' :'accepted_role_title',
    'booked_position': 'internship_position',
    'reserved_name': 'supervisor_name'
})

In [ ]:
import pandas as pd

booked_reserved_df = df_fix_participant_cleaned[[
    'booked_participant_name',
    'booked_skill',
    'booked_tools',
    'booked_role_title',
    'booked_position',
    'reserved_name'
    
]].copy()

booked_reserved_df = booked_reserved_df.rename(columns={
    'booked_participant_name': 'participant_name',
    'booked_skill': 'participant_skill',
    'booked_tools': 'participants_tools',
    'booked_role_title' :'accepted_role_title',
    'booked_position': 'internship_position',
    'reserved_name': 'supervisor_name'
})

df_fix_participant_cleaned = pd.concat([df_fix_participant_cleaned, booked_reserved_df], ignore_index=True)

df_fix_participant_cleaned = df_fix_participant_cleaned.drop(columns=[
    'booked_participant_name',
    'booked_skill',
    'booked_tools',
    'booked_role_title',
    'booked_position',
    'reserved_name'
])

print(df_fix_participant_cleaned)


In [ ]:
df_fix_participant_cleaned

In [ ]:
df_fix_participant_cleaned = df_fix_participant_cleaned[df_fix_participant_cleaned['status'] != 'ditolak']

In [ ]:
df_fix_participant_cleaned['status'] = df_fix_participant_cleaned['status'].fillna('ditolak')

In [ ]:
df_fix_participant_cleaned

In [ ]:
# Daftar kolom yang ingin di-drop
cols_to_drop = [
    'request_id', 'supervisor_nik', 'internship_role_id',
    'supervisor_id', 'participant_id', 'accepted_role_id',
    'reserve_id', 'booked_role_id'
]

# Drop kolom dari DataFrame
df_fix_participant_cleaned = df_fix_participant_cleaned.drop(columns=cols_to_drop)

In [ ]:
df_fix_participant_cleaned

In [ ]:

df_fix_participant_cleaned = df_fix_participant_cleaned.rename(columns={
    'supervisor_name': 'spv_mentee',
    'participant_name': 'mentee_name',
    'accepted_role_title': 'mentee_title',
    'participant_skill': 'mentee_skill',
    'participants_tools': 'mentee_tools',
    'internship_position': 'mentee_position',
    'internship_role_title': 'required_role_title',
    'status': 'mentee_status'
})


In [ ]:
df_fix_participant_cleaned

In [ ]:
total_sebelum = len(df_fix_participant_cleaned)

# Filter data
df_fix_participant_cleaned = df_fix_participant_cleaned[df_fix_participant_cleaned['mentee_position'] != '1. Tidak ada data']

# Hitung total baris setelah filter
total_setelah = len(df_fix_participant_cleaned)

# Hitung jumlah baris yang dihapus
baris_dihapus = total_sebelum - total_setelah

print(f"Jumlah baris yang dihapus: {baris_dihapus}")

In [ ]:
df_fix_participant_cleaned

In [ ]:
# Cari duplikat berdasarkan kolom 'booked_participant_name'
duplicates = df_fix_participant_cleaned[df_fix_participant_cleaned.duplicated(subset=['mentee_name'], keep=False)]

# Tampilkan duplikat
print("Daftar data yang terduplikasi di kolom 'booked_participant_name':")
print(duplicates[['mentee_name']])

# Hitung jumlah duplikat (unik yang muncul lebih dari 1x)
duplicate_count = df_fix_participant_cleaned['mentee_name'].value_counts()
duplicate_count = duplicate_count[duplicate_count > 1]

print("\nJumlah nama yang duplikat:", len(duplicate_count))
print("\nDetail jumlah kemunculan tiap nama yang duplikat:")
print(duplicate_count)


In [ ]:
# # Hapus duplikat pada kolom 'booked_participant_name', kecuali yang kosong
# df_fix_participant_cleaned = df_fix_participant_cleaned[df_fix_participant_cleaned['mentee_name'].notna() | 
#                                                          ~df_fix_participant_cleaned.duplicated(subset=['mentee_name'], keep='first')]

df_fix_participant_cleaned = df_fix_participant_cleaned.drop_duplicates(subset=['mentee_name'], keep='first')


In [ ]:
df_fix_participant_cleaned

In [ ]:
import pandas as pd
import os

# Lokasi output
output_loc = "*/transformed_src/"
output_file = os.path.join(output_loc, "df_fix_participant_cleaned.xlsx")

# Pastikan folder tujuan ada
if not os.path.exists(output_loc):
    os.makedirs(output_loc)

# Simpan ke Excel
df_fix_participant_cleaned.to_excel(output_file, index=False)


print(f"File berhasil disimpan di {output_file}")

In [ ]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# Load dataset
df_fix_participant_cleaned = pd.read_excel("*/transformed_src/df_fix_participant_cleaned.xlsx", engine="openpyxl")

# Daftar kolom yang ingin dibersihkan dan diatur wrapping text
columns_to_clean = df.columns  # Membersihkan semua kolom

def clean_text(text):
    if pd.isna(text):  # Jika NaN atau None, return kosong
        return ""
    
    text = str(text)  # Pastikan dalam bentuk string
    text = re.sub(r'[\[\]\(\){}<>]', '', text)  # Hapus tanda kurung [] () {} <>
    text = re.sub(r'[^a-zA-Z0-9.,:\- ]', '', text)  # Pertahankan karakter tertentu
    text = re.sub(r'\s+', ' ', text).strip()  # Hilangkan spasi berlebih
    
    return text.replace(',', '\n')  # Ubah koma menjadi baris baru

# Terapkan fungsi pembersihan pada semua kolom
df_fix_participant_cleaned = df_fix_participant_cleaned.applymap(clean_text)

# Simpan hasil sementara ke Excel
output_file = "*/transformed_src/data_fix_participant_cleaned.xlsx"
df_fix_participant_cleaned.to_excel(output_file, index=False, engine="openpyxl")

# Load file Excel untuk pengaturan wrapping text
wb = load_workbook(output_file)
ws = wb.active

# Atur lebar kolom agar sesuai dengan teks dan aktifkan wrap text
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)  # Dapatkan huruf kolom
    for cell in col:
        try:
            max_length = max(max_length, len(str(cell.value)))
            cell.alignment = Alignment(wrap_text=True)  # Aktifkan wrap text
        except:
            pass
    adjusted_width = min(max_length + 2, 50)  # Sesuaikan lebar kolom (max 50)
    ws.column_dimensions[col_letter].width = adjusted_width

# Simpan kembali dengan pengaturan wrapping text
wb.save(output_file)

print(f"File berhasil dibuat dengan teks yang lebih rapi.")


In [ ]:
df_fix_participant_cleaned

# Exploratory Data Analysis

In [ ]:
# === Import Library ===
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# === Load Data ===
df = pd.read_excel("*/transformed_src/data_fix_participant_cleaned.xlsx")  # ganti sesuai lokasi file

### Menghitung Mentee dan Mentor

In [ ]:
jumlah_stakeholder = {
    "Jumlah Unik Mentor": df['spv_mentee'].nunique(),
    "Jumlah Unik Mentee": df['mentee_name'].nunique(),
}

# Jumlah unik supervisor & mentee
jumlah_supervisor = df['spv_mentee'].nunique()
jumlah_mentee = df['mentee_name'].nunique()

print(f"Jumlah unik supervisor: {jumlah_supervisor}")
print(f"Jumlah unik mentee: {jumlah_mentee}")

### Perbandingan supervisor yang onjob vs tidak

In [ ]:
# Supervisor yang onjob-kan mentee
supervisor_onjob = df[df['mentee_status'] == 'diterima']['spv_mentee'].dropna().unique()

# Supervisor yang tidak onjob-kan mentee
supervisor_not_onjob = df[df['mentee_status'] == 'ditolak']['spv_mentee'].dropna().unique()

# Untuk menghindari tumpang tindih, kita ambil supervisor yang hanya ada di 'ditolak' dan tidak di 'diterima'
supervisor_murni_tidak_onjob = set(supervisor_not_onjob) - set(supervisor_onjob)

# Hitung jumlahnya
print(f"Jumlah supervisor yang onjob-kan mentee: {len(supervisor_onjob)}")
print(f"Jumlah supervisor yang tidak onjob-kan mentee: {len(supervisor_murni_tidak_onjob)}")

import matplotlib.pyplot as plt

labels = ['Supervisor Onjob', 'Supervisor Tidak Onjob']
values = [len(supervisor_onjob), len(supervisor_murni_tidak_onjob)]

plt.figure(figsize=(6, 4))
plt.bar(labels, values, color=['green', 'red'])
plt.title('Perbandingan Supervisor yang Onjob vs Tidak Onjob-kan Mentee')
plt.ylabel('Jumlah Supervisor')
plt.tight_layout()
plt.show()


### Daftar nama supervisor yang onjob

In [ ]:
# Ambil supervisor yang punya mentee dengan status diterima
supervisor_onjob = df[df['mentee_status'] == 'diterima']['spv_mentee'].dropna().unique()

# Ubah jadi list dan urutkan
supervisor_onjob_sorted = sorted(supervisor_onjob)

# Tampilkan
print("Supervisor yang onjob-kan mentee:")
for spv in supervisor_onjob_sorted:
    print("-", spv)


### Distribusi role yang tersedia vs onjob

In [ ]:
Jumlah_role = {
    "Jumlah Role Magang Tersedia": df['required_role_title'].nunique(),
    "Jumlah Role yang Onjob" : df['mentee_title'].nunique()
}

# Tampilkan distribusi role yang dibooking
print(df['required_role_title'].value_counts())
print("\n")

print(df['mentee_title'].value_counts())
print("\n")


# Histogram Role yang onjob
top_roles = df['mentee_title'].value_counts().head(10)
sns.barplot(x=top_roles.values, y=top_roles.index, palette='Blues_d')
plt.title('Top 10 Role yang Paling Banyak Onjob')
plt.xlabel('Jumlah Mentee Booked')
plt.ylabel('Role Onjob')
plt.show()






### Jumlah role mentee yang diterima oleh spv

In [ ]:
# Filter hanya mentee yang diterima
df_diterima = df[df['mentee_status'] == 'diterima']

# Hitung jumlah masing-masing mentee_title yang diterima
role_diterima_counts = df_diterima['mentee_title'].value_counts()

# Tampilkan hasilnya
print("Jumlah masing-masing role mentee_title yang diterima:")
print(role_diterima_counts)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
role_diterima_counts.plot(kind='barh', color='skyblue')
plt.title('Jumlah Role Mentee (mentee_title) yang Diterima')
plt.xlabel('Jumlah Diterima')
plt.ylabel('Role Mentee')
plt.tight_layout()
plt.gca().invert_yaxis()  # agar yang paling banyak ada di atas
plt.show()


### Top 5 role diterima

In [ ]:
# Filter hanya yang statusnya 'diterima'
df_diterima = df_fix_participant_cleaned[df_fix_participant_cleaned['mentee_status'] == 'diterima']

# Hitung jumlah masing-masing mentee_title
top_5_roles = df_diterima['mentee_title'].value_counts().head(5)

# Tampilkan hasilnya
print("Top 5 Role (mentee_title) yang Paling Banyak Diterima:")
print(top_5_roles)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.barplot(y=top_5_roles.index, x=top_5_roles.values, palette='Blues_d')
plt.title('Top 5 Role Mentee yang Paling Banyak Diterima')
plt.xlabel('Jumlah Diterima')
plt.ylabel('Role (mentee_title)')
plt.tight_layout()
plt.show()


### Menghitung perbandingan Mentee booked dan Onjob

In [ ]:
# Filter hanya mentee yang diterima
df_diterima = df[df['mentee_status'] == 'diterima']

# Hitung jumlah masing-masing mentee_position
mentee_position_counts = df_diterima['mentee_position'].value_counts()

# Tampilkan
print("Posisi Mentee (mentee_position) yang Paling Dibutuhkan Supervisor:")
print(mentee_position_counts)


### Perbandingan mentee yang booked vs onjob 

In [ ]:

# Hitung jumlah unik mentee
jumlah_dibooking = df_fix_cleaned['booked_participant_name'].nunique()
jumlah_onjob = df_fix_cleaned['participant_name'].nunique()

# Tampilkan
print("Jumlah mentee yang dibooking:", jumlah_dibooking)
print("Jumlah mentee yang onjob:", jumlah_onjob)

# Visualisasi perbandingan
plt.figure(figsize=(6, 4))
sns.barplot(x=['Booked', 'Onjob'], y=[jumlah_dibooking, jumlah_onjob], palette='Set2')
plt.title('Perbandingan Jumlah Mentee Booked vs Onjob')
plt.ylabel('Jumlah Unik Mentee')
plt.xlabel('Status')
plt.show()


### 	Mismatch role booked vs onjob

In [ ]:
# Kesesuaian Role dan Realisasi Booking
role_mismatch = df_fix_cleaned[df_fix_cleaned['internship_role_title'] != df_fix_cleaned['booked_role_title']]
print("Jumlah Mismatch Role:", len(role_mismatch))
print(role_mismatch['internship_role_title'].value_counts().head(10))

# Histogram Role Mismatch
mismatch_counts = role_mismatch['internship_role_title'].value_counts().head(10)
sns.barplot(x=mismatch_counts.values, y=mismatch_counts.index, palette='magma')
plt.title('Top 10 Role dengan Mismatch Terbanyak')
plt.xlabel('Jumlah Mismatch')
plt.ylabel('Internship Role')
plt.show()

### Top 5 skills & tools dimiliki mentee

In [ ]:
def extract_items(column_data):
    items = []
    for entry in column_data.dropna():
        split_items = [item.strip().lower() for item in entry.replace('\n', ',').split(',')]
        items.extend(split_items)
    return Counter(items)

# Skill & tools
top_participant_skill = extract_items(df['mentee_skill']).most_common(5)
top_participant_tools = extract_items(df['mentee_tools']).most_common(5)

print("Top 5 Skill yang dimiliki mentee:", top_participant_skill)
print("Top 5 Tools yang dimiliki mentee:", top_participant_tools)


# Visualisasi
def plot_bar_chart(data, title, color):
    labels, values = zip(*data)
    plt.figure(figsize=(8, 5))
    bars = plt.bar(labels, values, color=color)
    plt.title(title)
    plt.xlabel("Item")
    plt.ylabel("Jumlah")
    
    # Tambahkan angka di atas setiap batang
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f'{int(height)}', 
                 ha='center', va='bottom', fontsize=10)

    plt.ylim(0, max(values) + 5)
    plt.tight_layout()
    plt.show()

# Plot top 5 skill
plot_bar_chart(top_participant_skill, "Top 5 Skill yang Dimiliki Mentee", color='skyblue')

# Plot top 5 tools
plot_bar_chart(top_participant_tools, "Top 5 Tools yang Dimiliki Mentee", color='lightgreen')

### Top 5 skills & tools requested by supervisor

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Fungsi yang sudah kamu punya:
def extract_items(column_data):
    items = []
    for entry in column_data.dropna():
        split_items = [item.strip().lower() for item in entry.replace('\n', ',').split(',')]
        items.extend(split_items)
    return Counter(items)


# Skill & tools yang dibutuhkan supervisor
top_required_skill = extract_items(df['required_skills']).most_common(5)
top_required_tools = extract_items(df['required_tools']).most_common(5)

print("Top 5 Skill yang dibutuhkan supervisor:", top_required_skill)
print("Top 5 Tools yang dibutuhkan supervisor:", top_required_tools)


#Pisahkan data jadi 2 list: skill/tools dan frekuensinya
skills, skill_counts = zip(*top_required_skill)
tools, tool_counts = zip(*top_required_tools)

# Buat histogram (bar chart) untuk Skill
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)  # Subplot pertama untuk Skill
plt.bar(skills, skill_counts, color='skyblue')
plt.xlabel('Skill')
plt.ylabel('Frekuensi')
plt.title('Top 5 Skill yang Dibutuhkan Supervisor')
plt.xticks(rotation=45, ha='right')

# Buat histogram (bar chart) untuk Tools
plt.subplot(1, 2, 2)  # Subplot kedua untuk Tools
plt.bar(tools, tool_counts, color='salmon')
plt.xlabel('Tools')
plt.ylabel('Frekuensi')
plt.title('Top 5 Tools yang Dibutuhkan Supervisor')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

### Distribusi mentee_status (diterima vs ditolak)

In [ ]:
import matplotlib.pyplot as plt

# Hitung jumlah berdasarkan status
jumlah_status = df_fix_participant_cleaned['mentee_status'].value_counts()

# Plot histogram (bar chart)
plt.figure(figsize=(6, 4))
jumlah_status.plot(kind='bar', color=['green', 'red'])

# Tambahan info visual
plt.title('Jumlah Peserta Diterima vs Ditolak')
plt.xlabel('Status')
plt.ylabel('Jumlah Peserta')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Tampilkan nilai di atas bar
for i, val in enumerate(jumlah_status):
    plt.text(i, val + 1, str(val), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### Booking vs Onjob overlap

In [ ]:
import matplotlib.pyplot as plt

# Ambil nama mentee yang unik dan tidak kosong
booked = set(df_fix_cleaned['booked_participant_name'].dropna().unique())
onjob = set(df_fix_cleaned['participant_name'].dropna().unique())

# 1. Mentee yang booking dan juga on-job
booked_and_onjob = booked & onjob

# 2. Mentee yang booking tapi tidak on-job
booked_only = booked - onjob

# 3. Mentee yang on-job tapi tidak booking
onjob_only = onjob - booked

# Tampilkan jumlah
print(f"1. Booking & On-job        : {len(booked_and_onjob)} orang")
print(f"2. Booking tapi bukan On-job: {len(booked_only)} orang")
print(f"3. On-job tapi bukan Booking: {len(onjob_only)} orang")

# Histogram
labels = ['Booking & On-job', 'Booking Only', 'On-job Only']
counts = [len(booked_and_onjob), len(booked_only), len(onjob_only)]

plt.figure(figsize=(12, 8))
bars = plt.bar(labels, counts, color=['green', 'orange', 'blue'])

# Tambahkan label jumlah di atas tiap bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.5, int(yval), ha='center', va='bottom')

plt.title('Distribusi Mentee Booking vs On-job')
plt.ylabel('Jumlah Mentee')
plt.xlabel('Kategori')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Gabungkan semua skill menjadi satu string
required_skills = ','.join(df['required_skills'].dropna().tolist())
mentee_skill = ','.join(df['mentee_skill'].dropna().tolist())
mentee_tools = ','.join(df['mentee_tools'].dropna().tolist())
required_tools = ','.join(df['required_tools'].dropna().tolist())

# Buat WordCloud
wordcloud_A = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(required_skills)
wordcloud_B = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(mentee_skill)
wordcloud_C = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(mentee_tools)
wordcloud_D = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(required_tools)

# Tampilkan
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_A, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud: Required Skills dari Mentor")
plt.show()

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_B, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud: Skill dari Mentee")
plt.show()

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_C, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud: Tools dari Mentee")
plt.show()

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_D, interpolation='bilinear')
plt.axis('off')
plt.title("WordCloud: Required Tools dari Mentor")
plt.show()



# Data Prepocessing

In [ ]:
df_fix_participant_cleaned

#### Text formatting

In [ ]:
import re
import pandas as pd

# Hapus definisi variabel 'list' kalau pernah tertimpa
try:
    del list
except NameError:
    pass

# Fungsi untuk memecah teks panjang jadi list poin yang lebih rapi
def split_into_list(text):
    if not isinstance(text, str):
        return []
    text = re.sub(r'(?<=\d)\.(?=\S)', '. ', text)
    text = re.sub(r'(?<=-)(?=\S)', ' ', text)
    parts = re.split(r'(?:\d+\.\s*|-|\n)', text)
    cleaned = [part.strip() for part in parts if part.strip()]
    return cleaned

# Fungsi untuk ubah list jadi teks multiline
def list_to_multiline_text(lst):
    if not isinstance(lst, list):
        return ""
    return '\n'.join(lst)

# Terapkan ke dataframe kamu
df_fix_participant_cleaned['job_specification'] = df_fix_participant_cleaned['job_specification'].apply(
    lambda x: list_to_multiline_text(split_into_list(x))
)

df_fix_participant_cleaned['mentee_position'] = df_fix_participant_cleaned['mentee_position'].apply(
    lambda x: list_to_multiline_text(split_into_list(x))
)


In [ ]:
df_fix_participant_cleaned

### Lowering the Text

In [ ]:
df_lowercased = df_fix_participant_cleaned.applymap(
    lambda x: x.lower() if isinstance(x, str) else x
)

In [ ]:
df_lowercased

In [ ]:
columns_to_drop = [
    'job_specification'
]

df_lowercased.drop(columns=columns_to_drop, inplace=True)

### Removing remove non-word and non-whitespace characters

In [ ]:
df_lowercased = df_lowercased.replace(to_replace=r'[^\w\s]', value='', regex=True)

In [ ]:
df_lowercased

In [ ]:
import pandas as pd
import os

# Fungsi ubah list jadi multiline text dengan bullet
def list_to_multiline_text(lst):
    if isinstance(lst, list):
        return '\n'.join(f"• {item}" for item in lst)
    return lst

# Terapkan fungsi ke semua kolom di df_lowercased yang berisi list
for col in df_lowercased.columns:
    if df_lowercased[col].apply(lambda x: isinstance(x, list)).any():
        df_lowercased[col] = df_lowercased[col].apply(list_to_multiline_text)

# Lokasi penyimpanan
output_loc = "*/transformed_src/"
output_file = os.path.join(output_loc, "data_participant_preprocess_formatted.xlsx")

# Pastikan folder ada
os.makedirs(output_loc, exist_ok=True)

# Simpan ke Excel pakai xlsxwriter dan wrap text
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df_lowercased.to_excel(writer, sheet_name='Sheet1', index=False)

    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Format wrap text agar isi list tampil ke bawah dalam satu sel
    wrap_format = workbook.add_format({'text_wrap': True, 'valign': 'top'})

    # Terapkan format dan lebarkan semua kolom
    for idx, col in enumerate(df_lowercased.columns):
        worksheet.set_column(idx, idx, 30, wrap_format)

print(f"✅ File berhasil disimpan dengan rapi di: {output_file}")


# Data Transformation

In [ ]:
import os

output_loc = "*/transformed_src/"
output_path = os.path.join(output_loc, 'df_lowercased.csv')
df_lowercased.to_csv(output_path, index=False)

In [ ]:
df_lowercased

In [ ]:
import pandas as pd

# Load CSV asli
df_lowercased = pd.read_csv('*/app/data/mentee_history.csv')

# Copy dulu supaya df_lowercased tetap aman
df_new_mentee = df_lowercased.copy()

# Hapus kolom mentee_status di df_new_mentee
df_new_mentee.drop(columns=['mentee_status'], inplace=True)

# Simpan df_new_mentee ke CSV baru
df_new_mentee.to_csv('*/app/data/new_mentee.csv', index=False)

In [ ]:
df_new_mentee

### Transformation for TF-IDF Modeling

In [ ]:
from time import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

def size_mb(docs):
    return sum(len(s.encode("utf-8")) for s in docs) / 1e6

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt', download_dir='*/nltk_data_tfidf')
nltk.download('wordnet', download_dir='*/nltk_data_tfidf')
nltk.download('stopwords', download_dir='*/nltk_data_tfidf')
nltk.download('punkt_tab', download_dir='*/nltk_data_tfidf')

# Setelah itu, beri tahu NLTK lokasi tersebut
nltk.data.path.append('*/nltk_data_tfidf')

In [ ]:
def load_wbl_dataset(filepath, verbose=False):

    # Inisialisasi preprocessing tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def preprocess_text(text):
        text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
        tokens = nltk.word_tokenize(text)
        cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(cleaned_tokens)

    df = pd.read_csv(filepath)

    text_cols = [
        'spv_mentee', 'job_position', 'required_tools', 'required_skills', 
        'required_role_title', 'mentee_name', 'mentee_title', 'mentee_skill',
        'mentee_tools', 'mentee_position'
    ]
    df.fillna('', inplace=True)
    df['combined_text'] = df[text_cols].agg(' '.join, axis=1)

    # Terapkan preprocessing di sini, ke semua teks gabungan
    df['combined_text'] = df['combined_text'].apply(preprocess_text)

    y = df['mentee_status']
    target_names = y.unique()

    t0 = time()
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=2)
    X = vectorizer.fit_transform(df['combined_text'])
    feature_names = vectorizer.get_feature_names_out()
    duration = time() - t0

    if verbose:
        print(f"Dokumen: {len(df)}, Fitur: {X.shape[1]}")
        print(f"TF-IDF selesai dalam {duration:.2f} detik")

    return X, y, feature_names, target_names, df


In [ ]:
# def load_wbl_dataset(filepath, verbose=False):
#     """
#     Load and vectorize the WBL mentee-mentor matching dataset.
    
#     Param:
#         filepath (str): Path ke CSV file data kamu.
#         verbose (bool): Untuk menampilkan informasi proses.

#     Return:
#         X (TF-IDF matrix), y (labels), feature_names (list of terms), target_names (target labels), df (dataframe asli)
#     """
#     # 1. Load data kamu dari CSV
#     df = pd.read_csv(filepath)

#     # 2. Gabungkan semua kolom teks relevan jadi satu kolom 'combined_text'
#     text_cols = [
#         'spv_mentee', 'job_position', 'required_tools', 'required_skills', 
#         'required_role_title', 'mentee_name', 'mentee_title', 'mentee_skill',
#         'mentee_tools', 'mentee_position'
#     ]
#     df.fillna('', inplace=True)  # Hindari NaN
#     df['combined_text'] = df[text_cols].agg(' '.join, axis=1)

#     # 3. Target labels (misalnya 'mentee_status' untuk label target)
#     y = df['mentee_status']
#     target_names = y.unique()  # Mendapatkan nama kategori target (misalnya status mentee)
    
#     # 4. TF-IDF vectorization
#     t0 = time()
#     vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=2, stop_words='english')
#     X = vectorizer.fit_transform(df['combined_text'])
#     feature_names = vectorizer.get_feature_names_out()
#     duration = time() - t0

#     if verbose:
#         print(f"Dokumen: {len(df)}, Fitur: {X.shape[1]}")
#         print(f"TF-IDF selesai dalam {duration:.2f} detik")

#     return X, y, feature_names, target_names, df

In [ ]:
X, y, feature_names, target_names, df_raw = load_wbl_dataset("*/transformed_src/df_lowercased.csv", verbose=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Transformation for Word2Vec Modeling

In [ ]:
import pandas as pd
import os

# Load file yang sudah disimpan
file_path = "*/transformed_src/df_lowercased.csv"
df_lowercased = pd.read_csv(file_path)

# Gabungkan kolom-kolom menjadi satu kolom 'text'
df_lowercased['text'] = df_lowercased[['spv_mentee', 'job_position', 'required_tools', 
                                       'required_skills', 'required_role_title', 'mentee_name', 
                                       'mentee_title', 'mentee_skill', 'mentee_tools', 
                                       'mentee_position', 'mentee_status']
                                      ].astype(str).agg(' '.join, axis=1)

# Buat dataframe akhir untuk modeling
df_combined = df_lowercased[['text', 'mentee_status']].rename(columns={'mentee_status': 'labels'})

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_combined['labels'] = encoder.fit_transform(df_combined['labels'])

X = df_combined['text']
y = df_combined['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import nltk

# Hapus semua path sebelumnya
nltk.data.path.clear()

# Tambahkan path yang benar
nltk.data.path.append('*/nltk_data')

# Verifikasi path yang digunakan
print(nltk.data.path)

In [ ]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords

nltk.data.path.append('*/nltk_data')

nltk.download('punkt', download_dir='*/nltk_data')
nltk.download('stopwords', download_dir='*/nltk_data')
nltk.download('punkt_tab', download_dir='*/nltk_data')

# Fungsi preprocessing tanpa stemming
def preprocess(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Menghapus karakter non-alfabet
    text = re.sub("[^a-zA-Z]", " ", text)
    
    # Menurunkan semua huruf ke kecil
    text = text.lower()
    
    # Tentukan bahasa berdasarkan kehadiran kata-kata bahasa Indonesia atau Inggris
    stop_words_en = set(stopwords.words('english'))
    stop_words_id = set(stopwords.words('indonesian'))
    
    # Tentukan stopwords dan bahasa
    words = nltk.word_tokenize(text)
    language = 'id' if any(word in stop_words_id for word in words) else 'en'

    if language == 'en':
        stop_words = stop_words_en
    else:
        stop_words = stop_words_id
    
    # Tokenisasi dan hapus stopwords
    words = [word for word in words if word not in stop_words]
    
    return ' '.join(words)


In [ ]:
X_train = pd.Series(X_train)
X_test = pd.Series(X_test)

# Tidak usah isi NaN, biarkan tetap kosong
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [ ]:
from gensim.models import Word2Vec

VECTOR_SIZE = 100
WINDOW = 50
MIN_COUNT = 5

sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT)

In [ ]:
import numpy as np

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(VECTOR_SIZE)
    return np.mean(words_vecs, axis=0)

X_train_vec = np.array([vectorize(sentence) for sentence in X_train])
X_test_vec = np.array([vectorize(sentence) for sentence in X_test])

# Modelling dan Evaluation : Text Classification dengan TF-IDF dan Word2Vec

### Benchmarking Model Klasifikasi dengan Representasi TF-IDF dan 10-Fold Cross-Validation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

# Data
X = df_raw['combined_text']
y = df_raw['mentee_status']

# TF-IDF vektorisasi
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=2, stop_words="english")
X_tfidf = vectorizer.fit_transform(X)

# Daftar model yang akan diuji
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Ridge Classifier": RidgeClassifier(tol=1e-2, solver="sparse_cg"),
    "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Linear SVC": LinearSVC(max_iter=1000),
    "Log-loss SGD": SGDClassifier(loss="log_loss", max_iter=1000, tol=1e-3, random_state=42),
    "Nearest Centroid": NearestCentroid(),
    "Complement Naive Bayes": ComplementNB()
}

# Dictionary untuk menyimpan hasil seluruh model
all_results = {}

# 10-fold stratified CV
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Loop ke semua model
for model_name, clf in models.items():
    print(f"\nEvaluating: {model_name}")
    results = {
        "precision": [],
        "recall": [],
        "f1_score": []
    }

    for fold, (train_index, test_index) in enumerate(skf.split(X_tfidf, y), 1):
        X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        results["precision"].append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        results["recall"].append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        results["f1_score"].append(f1_score(y_test, y_pred, average='macro', zero_division=0))

        print(f"  Fold {fold}: f1={results['f1_score'][-1]:.3f}")

    # Simpan hasil rata-rata
    all_results[model_name] = {
        "Mean Precision": np.mean(results["precision"]),
        "Mean Recall": np.mean(results["recall"]),
        "Mean F1-Score": np.mean(results["f1_score"])
    }

# Tampilkan hasil akhir
print("\n" + "="*90)
print("Benchmarking Hasil Rata-Rata (10-Fold Cross-Validation):\n")

results_df = pd.DataFrame(all_results).T  # Transpose agar lebih rapi
print(results_df.round(3))


### Visualisasi Perbandingan Kinerja Model (10-Fold CV)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Urutkan data berdasarkan F1-Score tertinggi
results_df_sorted = results_df.sort_values(by="Mean F1-Score", ascending=False)

# Ambil list model dan metrik
models = results_df_sorted.index.tolist()
metrics = ["Mean Precision", "Mean Recall", "Mean F1-Score"]

# Data untuk setiap metrik
data = [results_df_sorted[metric].values for metric in metrics]

# Setup posisi bar
bar_width = 0.25
index = np.arange(len(models))

fig, ax = plt.subplots(figsize=(12, 6))

# Plot bar untuk tiap metrik per model
for i, metric in enumerate(metrics):
    ax.bar(index + i * bar_width, data[i], bar_width, label=metric)

# Label dan judul
ax.set_xlabel("Models")
ax.set_ylabel("Skor")
ax.set_title("Perbandingan Kinerja Model (10-Fold CV)")
ax.set_xticks(index + bar_width)
ax.set_xticklabels(models, rotation=45, ha="right")
ax.set_ylim(0, 1.05)  # agar ada ruang untuk label nilai

# Grid untuk memudahkan pembacaan
ax.grid(True, linestyle="--", alpha=0.6, axis='y')

# Tambahkan nilai di atas setiap bar
for i in range(len(models)):
    for j in range(len(metrics)):
        ax.text(index[i] + j*bar_width, data[j][i] + 0.02, f"{data[j][i]:.3f}",
                ha='center', va='bottom', fontsize=8)

ax.legend()
plt.tight_layout()
plt.show()


### Benchmarking Model Klasifikasi dengan Representasi Word2Vec dan 10-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from time import time

# Daftar model
models = {
    "Logistic Regression": LogisticRegression(C=5, max_iter=1000),
    "Ridge Classifier": RidgeClassifier(alpha=1.0, solver="sparse_cg"),
    "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Linear SVC": LinearSVC(C=0.1, dual=False, max_iter=1000),
    "Log-loss SGD": SGDClassifier(loss="log_loss", alpha=1e-4, n_iter_no_change=3, early_stopping=True),
    "Nearest Centroid": NearestCentroid()
}

# Cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_results = {}

X_clean = X.apply(preprocess)
X_w2v = np.array([vectorize(sentence) for sentence in X_clean])


# Uji semua model
for model_name, clf in models.items():
    print(f"\nEvaluating: {model_name}")
    results = {
        "precision": [],
        "recall": [],
        "f1_score": [],
        "train_time": [],
        "test_time": []
    }

    for fold, (train_idx, test_idx) in enumerate(skf.split(X_w2v, y), 1):
        X_train, X_test = X_w2v[train_idx], X_w2v[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        t0 = time()
        clf.fit(X_train, y_train)
        train_duration = time() - t0

        t0 = time()
        y_pred = clf.predict(X_test)
        test_duration = time() - t0

        prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
        rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

        results["precision"].append(prec)
        results["recall"].append(rec)
        results["f1_score"].append(f1)
        results["train_time"].append(train_duration)
        results["test_time"].append(test_duration)

        print(f"  f1={f1:.3f}")

    # Simpan rata-rata hasil
    all_results[model_name] = {
        "Mean Precision": np.mean(results["precision"]),
        "Mean Recall": np.mean(results["recall"]),
        "Mean F1-Score": np.mean(results["f1_score"]),
        "Avg Train Time": np.mean(results["train_time"]),
        "Avg Test Time": np.mean(results["test_time"]),
    }

# Tampilkan hasil akhir dalam bentuk DataFrame
print("\n" + "="*90)
print("Benchmarking Hasil Rata-Rata (Word2Vec + 10-Fold Cross-Validation):\n")

results_df = pd.DataFrame(all_results).T
print(results_df.round(3))


### Visualisasi Perbandingan Kinerja Model (10-Fold CV)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ambil data hasil benchmark
results_df = pd.DataFrame(all_results).T

# Urutan model dan metrik
models = results_df.index.tolist()
metrics = ['Mean Precision', 'Mean Recall', 'Mean F1-Score']

# Data untuk setiap metrik
data = [results_df[metric].values for metric in metrics]

# Setup posisi bar
bar_width = 0.25
index = np.arange(len(models))

# Membuat figure dan axis
fig, ax = plt.subplots(figsize=(14, 6))

# Plot tiap metrik sebagai bar yang berdampingan
for i, metric in enumerate(metrics):
    ax.bar(index + i * bar_width, data[i], bar_width, label=metric)

# Label dan judul
ax.set_xlabel('Models')
ax.set_ylabel('Score')
ax.set_title('Perbandingan Kinerja Model (10-Fold CV)')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(models, rotation=45, ha='right')
ax.set_ylim(0, 1.05)  # supaya ada ruang di atas

# Tambahkan legend
ax.legend()

# Tampilkan nilai di atas setiap bar
for i in range(len(models)):
    for j in range(len(metrics)):
        ax.text(index[i] + j*bar_width, data[j][i] + 0.02, f"{data[j][i]:.3f}", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

colors = ['skyblue'] * len(results_df)

# Visualisasi waktu training dan testing (opsional)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training time
axes[0].barh(results_df.index, results_df['Avg Train Time'], color=colors)
axes[0].set_title('Average Training Time (seconds)')
axes[0].invert_yaxis()
for idx, val in enumerate(results_df['Avg Train Time']):
    axes[0].text(val + max(results_df['Avg Train Time'])*0.01, idx, f"{val:.3f}", va='center')

# Testing time
axes[1].barh(results_df.index, results_df['Avg Test Time'], color=colors)
axes[1].set_title('Average Testing Time (seconds)')
axes[1].invert_yaxis()
for idx, val in enumerate(results_df['Avg Test Time']):
    axes[1].text(val + max(results_df['Avg Test Time'])*0.01, idx, f"{val:.3f}", va='center')

plt.suptitle('Comparison of Model Computational Time', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# Analisis Evaluasi Model

#### Benchmark tiap model dengan TF-IDF:

| Model                 | Mean Precision | Mean Recall | Mean F1-Score |
|-----------------------|----------------|-------------|---------------|
| Logistic Regression    | 0.973          | 0.970       | 0.970         |
| **Ridge Classifier**       | 0.981          | 0.977       | **0.978**         |
| k-Nearest Neighbors    | 0.929          | 0.909       | 0.911         |
| **Random Forest**          | 0.982          | 0.982       | **0.981**         |
| Linear SVC             | 0.973          | 0.970       | 0.970         |
| Log-loss SGD           | 0.976          | 0.972       | 0.973         |
| Nearest Centroid       | 0.932          | 0.932       | 0.930         |
| Complement Naive Bayes | 0.960          | 0.956       | 0.957         |



#### Benchmark tiap model dengan Word2Vec:

| Model                 | Precision | Recall | F1-Score | Avg Train Time (s) | Avg Test Time (s) |
|-----------------------|----------------|-------------|---------------|--------------------|-------------------|
| Logistic Regression    | 0.672          | 0.651       | 0.646         | 0.005              | 0.000             |
| Ridge Classifier       | 0.667          | 0.641       | 0.633         | 0.002              | 0.000             |
| k-Nearest Neighbors    | 0.729          | 0.725       | 0.724         | 0.001              | 0.001             |
| **Random Forest**          | 0.835          | 0.822       | **0.822**         | 0.278              | 0.006             |
| Linear SVC             | 0.724          | 0.616       | 0.578         | 0.002              | 0.000             |
| Log-loss SGD           | 0.350          | 0.503       | 0.343         | 0.005              | 0.000             |
| Nearest Centroid       | 0.625          | 0.621       | 0.619         | 0.001              | 0.001             |


### 📊 Analisis Evaluasi Model

Berdasarkan hasil benchmark dengan dua jenis representasi fitur — **TF-IDF** dan **Word2Vec** — diperoleh beberapa poin penting sebagai berikut:

**1. Random Forest menunjukkan performa terbaik secara konsisten**
- Pada representasi **TF-IDF**, Random Forest mencatat skor F1 tertinggi (**0.981**), sedikit lebih tinggi dari Ridge Classifier (**0.978**).
- Pada **Word2Vec**, Random Forest tetap menjadi model terbaik dengan F1-Score **0.822**, unggul jauh dibanding model lainnya.

**2. Ridge Classifier memberikan hasil yang kompetitif dengan TF-IDF**
- Ridge Classifier menjadi alternatif kuat untuk TF-IDF, dengan performa tinggi serta waktu pelatihan dan pengujian yang sangat efisien.

**3. Kinerja model cenderung menurun pada Word2Vec**
- Sebagian besar model mengalami penurunan skor F1 ketika menggunakan Word2Vec, termasuk Logistic Regression dan Linear SVC.
- Hal ini kemungkinan disebabkan oleh jumlah data yang terbatas (sekitar 300 data) serta karakteristik teks yang pendek.
- TF-IDF yang hanya melihat seberapa sering kata muncul ternyata lebih cocok untuk kondisi ini, karena model seperti Logistic Regression dan Linear SVC jadi lebih mudah menangkap pola dari data yang sudah rapi dan terstruktur dengan baik.

**4. Keterbatasan Word2Vec dalam konteks data kecil**
- Word2Vec bekerja dengan baik jika dilatih pada dataset besar karena perlu memahami relasi antar kata secara kontekstual.
- Dalam kasus ini, meskipun telah menggunakan versi pre-trained, model tersebut belum tentu sesuai dengan konteks spesifik dataset, sehingga representasi katanya tidak optimal dan menyebabkan performa model menjadi lebih rendah dibanding TF-IDF.


# **Buat dan Simpan Model**

In [ ]:
import joblib
import os

# Buat ulang model Random Forest di seluruh data
final_model = RandomForestClassifier(n_estimators=100, random_state=42)
final_model.fit(X_tfidf, y)

# Simpan model dan vectorizer
os.makedirs('*/model', exist_ok=True)

joblib.dump(final_model, '*/model/matching-model.pkl')
joblib.dump(vectorizer, '*/model/tf-idfvectorizer.pkl')

print("✅ Model dan TF-IDF vectorizer berhasil disimpan.")


# **Inspect Model**

### Inspect TF-IDF Vectorizer

In [ ]:
import joblib

# Load the vectorizer
vectorizer = joblib.load('*/model/tf-idfvectorizer.pkl')

# Tampilkan tipe dan info umum
print("Tipe model vectorizer:", type(vectorizer))
print("Jumlah kosakata (vocabulary):", len(vectorizer.vocabulary_))

# Contoh item dari vocabulary
print("Contoh vocabulary:", list(vectorizer.vocabulary_.items())[:10])

# Nilai IDF
print("Nilai IDF:", vectorizer.idf_[:10])

# Coba vektorisasi kalimat contoh
sample_text = ["pengembangan sistem informasi"]
sample_text = list(map(lambda x: x.lower(), sample_text))  # lowercase
features = vectorizer.transform(sample_text)

# TF-IDF hasil (versi dense matrix)
print("TF-IDF (dense):", features.todense())

# Fitur nama
feature_names = vectorizer.get_feature_names_out()
print("Nama fitur:", feature_names[:10])


In [ ]:
# Load model
model = joblib.load('*/model/matching-model.pkl')

# Tipe model
print("Tipe model:", type(model))

# Parameter model
print("Parameter:", model.get_params())

# Informasi tambahan jika tersedia
if hasattr(model, 'n_features_in_'):
    print("Jumlah fitur input:", model.n_features_in_)

if hasattr(model, 'classes_'):
    print("Label Kelas:", model.classes_)


In [ ]:
# Sample input (disamakan formatnya)
sample_text = ["pengembangan sistem informasi"]
sample_text = list(map(lambda x: x.lower(), sample_text))
features = vectorizer.transform(sample_text)

# Prediksi probabilitas
pred_prob = model.predict_proba(features)

# Tampilkan sebagai DataFrame dengan label kelas
import pandas as pd
label_map = ['Data Scientist', 'Designer', 'Developer', 'Engineer', 'General', 'Marketing', 'Researcher']
df_pred = pd.DataFrame(pred_prob, columns=label_map)
print(df_pred)


# **Kesimpulan**

Random Forest dengan representasi TF-IDF adalah model terbaik untuk sistem rekomendasi mentee-role WBL, dengan F1-Score 0.981. Model ini menunjukkan akurasi tinggi, konsistensi precision dan recall.